In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [ ]:
import multiprocessing
from torch import autograd
from fastai.conv_learner import *
from fasterai.images import *
from fasterai.dataset import *
from fasterai.visualize import *
from fasterai.loss import *
from fasterai.modules import *
from fasterai.wgan import *
from fasterai.generators import *
from fastai.torch_imports import *
from pathlib import Path
from itertools import repeat
import tensorboardX
torch.cuda.set_device(2)
plt.style.use('dark_background')
torch.backends.cudnn.benchmark=True


In [ ]:
DATA_PATH = Path('data/imagenet/ILSVRC/Data/CLS-LOC')
TRAIN_SOURCE_PATH = DATA_PATH/'train'
proj_id = 'bw2color'
TENSORBOARD_PATH = Path('data/tensorboard/UnetWithCombinedLoss')

dpath = DATA_PATH/(proj_id + '_8989_D.h5')
gpath = DATA_PATH/(proj_id + '_8989_G.h5')
keep_pct=0.1
bs=32
sz=224
wd=1e-7
lr=5e-2
lrs = np.array([lr/10000,lr/1000,lr])

torch.backends.cudnn.benchmark=True

In [ ]:
class CombinedLoss(nn.Module):		
    def __init__(self, multiplier:float=1.0):
        super().__init__()
        self.ploss = PerceptualLoss()
        self.floss = FeatureLoss(multiplier=1e3)
	
    def forward(self, input, target):
        p = self.ploss(input, target)
        f = self.floss(input, target)
        return p+f

## Training

In [ ]:
md = get_matched_image_model_data(image_size=sz, batch_size=bs, root_data_path=DATA_PATH, train_root_path=TRAIN_SOURCE_PATH, proj_id=proj_id, keep_pct=keep_pct)

In [ ]:
netG = Unet34(nf_factor=2).cuda()
#netGVis = ModelStatsVisualizer(TENSORBOARD_PATH, netG, 'netG')
unet = LearnerGenModuleWrapper(netG, 'unet')
#load_model(netG, gpath)
learn = ConvLearner(md, unet)
learnVisCB = ImageGenLearnerVisualizerCB(TENSORBOARD_PATH, netG, md, 'learner')
learn.metrics = []
learn.opt_fn=optim.Adam
#learn.crit = F.mse_loss #(turns sepia/blurry)
learn.crit = CombinedLoss()

In [ ]:
#learn.freeze_to(1)

In [ ]:
#learn.lr_find(1e-4, 1e1, wds=wd, linear=False)

In [ ]:
#learn.sched.plot(n_skip=0, n_skip_end=0)

In [ ]:
learn.fit(lrs, 1, cycle_len=2, wds=wd, use_clr_beta=(20,10,0.95,0.85), callbacks=[learnVisCB])